In [1]:
import os
import random
import numpy as np
import matplotlib as plt
from IPython.display import HTML

# Pytorch and Torchvision Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.optim import Adam
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutils

# Torchgan Imports
import torchgan.models as models
import torchgan.losses as losses
from torchgan.trainer import Trainer

In [2]:
#set random seed
seed = 999
random.seed(seed)
torch.manual_seed(seed)
print("Random Seed: ", seed)

Random Seed:  999


Load Datasets

In [5]:
dataset = dsets.MNIST(
    root = "./mnist",
    train = True,
    transform=transforms.Compose(
        [
            transforms.Resize((32,32)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,),std=(0.5,)),
        ]
    ),
    download=True,
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist\MNIST\raw



In [6]:
loader = data.DataLoader(dataset,batch_size=64,shuffle=True)

Generator Model

In [7]:
class AdversarialAutoencoderGenerator(models.Generator):
    def __init__(
            self,
            encoding_dims,
            input_size,
            input_channels,
            step_channels=16,
            nonlinearity=nn.LeakyReLU(0.2),
    ):
        super(AdversarialAutoencoderGenerator, self).__init__(encoding_dims)
        encoder = [nn.Sequential(nn.Conv2d(input_channels, step_channels, 5, 2 ,2), nonlinearity
                                 )
                            ]
        
        size = input_size // 2
        channels = step_channels
        while size > 1:
            encoder.append(
                nn.Sequential(
                    nn.Conv2d(channels, channels * 4, 5, 4, 2),
                    nn.BatchNorm2d(channels * 4),
                    nonlinearity,
                )
            )
            channels *= 4
            size = size // 4

            self.encoder = nn.Sequential(*encoder)
            self.encoder_fc = nn.Linear(
                channels, encoding_dims
            )
            self.decoder_fc = nn.Linear(encoding_dims, step_channels)
            decoder = []
            size = 1
            channels = step_channels
            while size < input_size // 2:
                decoder.append(
                    nn.Sequential(
                        nn.ConvTranspose2d(channels, channels * 4, 5, 4, 2, 3),
                        nn.BatchNorm2d(channels * 4),
                        nonlinearity,
                    )
                )
            
            channels *= 4
            size *= 4
            decoder.append(nn.ConvTranspose2d(channels, input_channels, 5, 2, 2, 1))
            self.decoder = nn.Sequential(*decoder)

    def sample(self, noise):
        noise = self.decoder_fc(noise)
        noise = noise.view(-1, noise.size(1), 1, 1)
        return self.decoder(noise)
    
    def forward(self, x):
        if self.training:
            encoding = self.encoder(x)
            encoding = self.encoder_fc(
                encoding.view(
                    -1, encoding.size(1) * encoding.size(2) * encoding.size(3)
                )
            )
            return self.sample(encoding), encoding
        else:
            return self.sample(x)
        


Disciminiator Network

In [ ]:
class AdversarialAutoencoderDiscriminator(models.Discriminator):
    def __init__(self, input_dims, nonlinearity = nn.LeakyReLU(0.2)):
        super(AdversarialAutoencoderDiscriminator, self).__init__(input_dims)
        model = [nn.Sequential(nn.Linear(input_dims, input_dims // 2), nonlinearity)]
        size = input_dims // 2
        while size > 16:
            model.append(
                nn.Sequential(
                    nn.Linear(size, size // 2), nn.BatchNorm1d(size // 2), nonlinearity
                )
            )
        size = size // 2
        model.append(nn.Linear(size, 1))
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

Loss Functions
1. Reconstruction error: MSE between input signal and reconstruction
2. Negative log likelihood of generated noise wrt discriminator

In [8]:
class AdversarialAutoencoderGeneratorLoss(losses.GeneratorLoss):
    def forward(self, real_inputs, gen_inputs, dgz):
        loss = 0.999 * F.mse_loss(gen_inputs, real_inputs)
        target = torch.ones_like(dgz)
        loss += 0.001 * F.binary_cross_entropy_with_logits(dgz, target)
        return loss

    def train_ops(
        self,
        generator,
        discriminator,
        optimizer_generator,
        real_inputs,
        device,
        batch_size,
        labels=None,
    ):
        recon, encodings = generator(real_inputs)
        optimizer_generator.zero_grad()
        dgz = discriminator(encodings)
        loss = self.forward(real_inputs, recon, dgz)
        loss.backward()
        optimizer_generator.step()
        return loss.item()


class AdversarialAutoencoderDiscriminatorLoss(losses.DiscriminatorLoss):
    def forward(self, dx, dgz):
        target_real = torch.ones_like(dx)
        target_fake = torch.zeros_like(dx)
        loss = 0.5 * F.binary_cross_entropy_with_logits(dx, target_real)
        loss += 0.5 * F.binary_cross_entropy_with_logits(dgz, target_fake)
        return loss

    def train_ops(
        self,
        generator,
        discriminator,
        optimizer_discriminator,
        real_inputs,
        device,
        batch_size,
        labels=None,
    ):
        _, encodings = generator(real_inputs)
        noise = torch.randn(real_inputs.size(0), generator.encoding_dims, device=device)
        optimizer_discriminator.zero_grad()
        dx = discriminator(noise)
        dgz = discriminator(encodings)
        loss = self.forward(dx, dgz)
        loss.backward()
        optimizer_discriminator.step()
        return loss.item()


losses = [
    AdversarialAutoencoderGeneratorLoss(),
    AdversarialAutoencoderDiscriminatorLoss(),
]

: 

In [3]:
import h5py

filename = 'model_application.h5'
h5 = h5py.File(filename,'r')
for item in h5.keys():
    print(item + ":", h5[item])

model_weights: <HDF5 group "/model_weights" (21 members)>
optimizer_weights: <HDF5 group "/optimizer_weights" (1 members)>


In [4]:
for item in h5.require_group('model_weights').keys():
    print(item)

conv1d
conv1d_1
conv1d_2
conv1d_3
conv1d_4
conv1d_5
conv1d_6
conv1d_transpose
conv1d_transpose_1
conv1d_transpose_2
conv1d_transpose_3
conv1d_transpose_4
conv1d_transpose_5
input_1
max_pooling1d
max_pooling1d_1
max_pooling1d_2
top_level_model_weights
up_sampling1d
up_sampling1d_1
up_sampling1d_2
